In [113]:
import sys

sys.path.append("../")

import pandas as pd
import numpy as np
import datetime
import os
from pprint import pprint
import matplotlib.pyplot as plt
import time
import vectorbtpro as vbt
from time import time
import helpers as pth
import platform
from dotenv import load_dotenv
import scipy.stats as stats
import time
import helpers as pth
from numba import njit
import talib

theme = "light"
vbt.settings.set_theme(theme)
# optional
# vbt.settings["plotting"]["layout"]["width"] = 640
# vbt.settings["plotting"]["layout"]["height"] = 320

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 20)
# plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12, 7)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams["axes.formatter.limits"] = [-1000000000, 1000000000]
plt.style.use("classic" if theme == "light" else "dark_background")

if platform.system().lower() == "windows":
    base_data_path = "H:\\phitech-data\\01_raw"
else:
    from core_chains.simple.llm import make_Q_chain

    base_data_path = "../../phitech-data/01_raw"
    load_dotenv("../../sandatasci-core/credentials")
    Q = make_Q_chain("gpt-4o-instance1", __vsc_ipynb_file__)

In [114]:
df = pth.SierraChartData.pull(
    ['MES'],
    start='2024-08-01',
    end='2024-08-02',
).resample('1min')
close = df.get('close')
close.vbt.plot()

FigureWidget({
    'data': [{'name': 'MES',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5a40f37f-02e5-4b56-932e-93a5ae12fbca',
              'x': array([datetime.datetime(2024, 8, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 2, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 8, 1, 23, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 23, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 23, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([5767.  , 5767.5 , 5767.75, ..., 5635.25, 5635.25, 5635.5 ])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
   

### from_order_func

In [115]:
import numpy as np
import pandas as pd
from numba import njit
import vectorbtpro as vbt

# @njit
def order_func_nb(c):
    if c.i % 10 == 0:
        return vbt.pf_nb.order_nb(size=1)
    if c.i % 15 == 0:
        return vbt.pf_nb.order_nb(size=-1)
    return vbt.pf_enums.NoOrder


pf = vbt.Portfolio.from_order_func(
    close[:100],
    order_func_nb=order_func_nb,
    jitted=False
)

pf.plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'mode': 'lines',
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'b96224b7-6ea1-4c1b-8c99-c225facd9aae',
              'x': array([datetime.datetime(2024, 8, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 1, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 2, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 3, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 4, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 5, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 8, 1, 0, 6, tzinfo=datetime.timezone.utc),
                          datetime.datetime(20

In [116]:
pf.orders.readable

,Order Id,Column,Index,Size,Price,Fees,Side
0,0,MES,2024-08-01 00:00:00+00:00,0.017340,5767.00,0.0,Buy
1,1,MES,2024-08-01 00:15:00+00:00,0.034680,5771.25,0.0,Sell
2,2,MES,2024-08-01 00:20:00+00:00,0.034679,5771.50,0.0,Buy
3,3,MES,2024-08-01 00:45:00+00:00,0.034677,5772.50,0.0,Sell
4,4,MES,2024-08-01 00:50:00+00:00,0.034680,5772.00,0.0,Buy
5,5,MES,2024-08-01 01:15:00+00:00,0.034683,5776.25,0.0,Sell
6,6,MES,2024-08-01 01:20:00+00:00,0.034689,5775.25,0.0,Buy


### Dynamic Stop loss

In [156]:
from collections import namedtuple


# Memory = namedtuple("Memory", ["counter"])
# init_memory = lambda target_shape: Memory(
#     counter=np.full(target_shape[1], np.nan)
# )


# @njit(nogil=True)
def signal_func(c, fast, slow):
    if not vbt.pf_nb.in_long_position_nb(c):
        if vbt.pf_nb.iter_crossed_above_nb(c, fast, slow):
            return pth.LONG_ENTER()

    sl_info = c.last_sl_info[c.col]
    if vbt.pf_nb.in_long_position_nb(c):
        position =  c.last_pos_info[c.col]
        entry_price = position['entry_price']
        if not vbt.pf_nb.is_stop_info_active_nb(sl_info):
            vbt.pf_nb.set_sl_info_nb(
                sl_info,
                init_price=entry_price,
                init_idx=c.i,
                stop=entry_price - 1.0,
                delta_format=vbt.pf_enums.DeltaFormat.Target,
                exit_size_type=vbt.pf_enums.SizeType.Amount,
                exit_size=position['size'],
            )
        else:
            # modify existng stop loss based on pnl
            if position['pnl'] > 50:
                vbt.pf_nb.set_sl_info_nb(
                    sl_info,
                    init_price=entry_price,
                    init_idx=c.i,
                    stop=fast[c.i, c.col],
                    delta_format=vbt.pf_enums.DeltaFormat.Target,
                    exit_size_type=vbt.pf_enums.SizeType.Amount,
                    exit_size=position['size'],
                )

    # if no positions -> remove all stops and targets
    if vbt.pf_nb.get_n_active_positions_nb(c) == 0:
        vbt.pf_nb.clear_sl_info_nb(c.last_sl_info[c.col])
        vbt.pf_nb.clear_tp_info_nb(c.last_tp_info[c.col])
        pass

    return pth.NO_SIGNAL()


n = 10000
fast = vbt.talib('MA').run(close[:n], 10)
slow = vbt.talib('MA').run(close[:n], 30)

pf = vbt.Portfolio.from_signals(
    close[:n],
    signal_func_nb=signal_func,
    signal_args=(
        # vbt.RepFunc(init_memory),
        vbt.Rep('fast'),
        vbt.Rep('slow'),
    ),
    broadcast_named_args=dict(
        fast=fast,
        slow=slow,
    ),
    size=1.0,
    size_granularity=1,
    size_type=vbt.pf_enums.SizeType.Percent,
    init_cash=1000000,
    jitted=False,
)
fig = pf.plot()
fast.ma[:n].vbt.plot(fig=fig)
slow.ma[:n].vbt.plot(fig=fig)
fig.show()

In [159]:
pf.positions.readable.head()

,Position Id,Column,Size,Entry Order Id,Entry Index,Avg Entry Price,Entry Fees,Exit Order Id,Exit Index,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status
0,0,"(10, 30, MES)",173.0,0,2024-08-01 01:05:00+00:00,5774.25,0.0,1,2024-08-01 01:18:00+00:00,5775.475,0.0,211.925,0.000212,Long,Closed
1,1,"(10, 30, MES)",173.0,2,2024-08-01 02:24:00+00:00,5769.50,0.0,3,2024-08-01 02:28:00+00:00,5768.500,0.0,-173.000,-0.000173,Long,Closed
2,2,"(10, 30, MES)",173.0,4,2024-08-01 03:03:00+00:00,5766.25,0.0,5,2024-08-01 03:06:00+00:00,5765.250,0.0,-173.000,-0.000173,Long,Closed
3,3,"(10, 30, MES)",173.0,6,2024-08-01 03:09:00+00:00,5767.50,0.0,7,2024-08-01 03:11:00+00:00,5766.500,0.0,-173.000,-0.000173,Long,Closed
4,4,"(10, 30, MES)",173.0,8,2024-08-01 03:21:00+00:00,5769.25,0.0,9,2024-08-01 03:30:00+00:00,5771.175,0.0,333.025,0.000334,Long,Closed
